In [3]:
def run_query(query, database, s3_output):
    client_athena = boto3.client('athena', region_name = 'us-east-1')
    response = client_athena.start_query_execution(
        QueryString=query,
        QueryExecutionContext={
            'Database': database
            },
        ResultConfiguration={
            'OutputLocation': s3_output,
            }
        )
    return response

In [4]:
import boto3
import pandas as pd
import pyarrow
import pandasglue as pg
import dask.dataframe as dd
pd.__version__

'0.24.1'

In [11]:
%%time
# FORMAT PARQUET FROM PANDAS
client_s3 = boto3.client('s3')
s3 = boto3.resource('s3')
bucket_name = 'attemp-glue'
bucket_csv = s3.Bucket(bucket_name)
marca = 'gestion.pe'

key_input = '{0}/2018/12'.format(marca)
bucket_output = 'athena-attemp-2'


"""
# CREO UN DATAFRAME CON TODOS LOS DIAS DEL MES
all_data_df = dd.read_csv(url_up)
all_data_df.head()
url_up = 'https://s3-us-west-1.amazonaws.com/{}/{}/*.csv'.format(bucket_name, key_input)
print(url_up)
"""

# CREO UN DATAFRAME CON TODOS LOS DIAS DEL MES
list_df = []
for obj in bucket_csv.objects.filter(Prefix=key_input):
    key = obj.key
    url_up = 'https://s3-us-west-1.amazonaws.com/{}/{}'.format(bucket_name, key)
    list_df.append(
        pd.read_csv(url_up)
    )

all_data_df = pd.concat(list_df)
del list_df

CPU times: user 841 ms, sys: 91 ms, total: 932 ms
Wall time: 18.5 s


In [15]:
print(all_data_df.shape)
all_data_df.head()

(178242, 3)


,dia,page_avg_time,mes
0,2018-12-18,61,12
1,2018-12-18,44,12
2,2018-12-18,0,12
3,2018-12-18,67,12
4,2018-12-18,15,12


In [16]:
# PASO EL CONTENIDO DE MI DATAFRAME A FORMATO 
# PARQUET PARTICIONADO POR LA COLUMNA DIA EN COMPRESION SNAPPY

import pandas as pd
import pandasglue as pg

#Parameters
database = 'athenatest'
table = 'charbeat'
s3_input = 's3://{0}/{1}'.format(bucket_output, marca)

pg.write_glue(all_data_df, database, table, s3_input, partition_cols=['dia'])

OSError: Write failed: charbeat/dia=2018-12-18/d4533a6fa4de4407b16833004761d5c6.parquet

In [53]:
%%time
import time

bucket_db = 'athena-db-table'

s3_input = 's3://{0}/{1}'.format(bucket_output, marca)
s3_ouput = 's3://{0}'.format(bucket_db)

print(s3_input, s3_ouput)
database = 'athenatest'
table = 'charbeat'

create_database = "CREATE DATABASE IF NOT EXISTS {0};".format(database)
res_1 = run_query(create_database, database, s3_ouput)
res_1

s3://athena-attemp/gestion.pe s3://athena-db-table
Execution ID: 2ce69cae-7ad2-44c2-af9c-4e7e498ecd09
CPU times: user 46.6 ms, sys: 3.01 ms, total: 49.6 ms
Wall time: 564 ms


In [57]:
database, table

('athenatest', 'charbeat')

In [67]:
create_table = \
"""CREATE EXTERNAL TABLE {}
('dia' STRING, 'page_avg_time' STRING, 'mes' STRING)
PARTITIONED BY (dia STRING)
LOCATION '{}'
STORED AS PARQUET;
""".format(table, s3_input)

res = run_query(create_table, database, s3_ouput)
print(res)

InvalidRequestException: An error occurred (InvalidRequestException) when calling the StartQueryExecution operation: line 1:8: no viable alternative at input 'CREATE EXTERNAL'